# Interactively Browse CMR Records for ABoVE Collections & Granules

NASA's **Common Metadata Repository (CMR)** is a metadata system that catalogs all data and service metadata records for NASA's Earth Observing System Data and Information System (EOSDIS). This notebook is for browsing the metadata records for ABoVE **Collections** and **Granules**, EOSDIS lingo for datasets and data files. Read more about CMR here:             
https://earthdata.nasa.gov/about/science-system-description/eosdis-components/common-metadata-repository

This notebook doesn't rely on live queries to CMR. Rather, it references a few tables that were generated beforehand and saved to binary files using Python's `pickle` module:
* **ABoVE Collections** (data/above_dataset_table.pkl): dataset records in CMR as 2019-05-12
* **ABoVE Granules** (data/above_granules_table.pkl): data file records in CMR as 2019-05-12
* **ABoVE Grid Table** (data/above_grid_table_ab.pkl): table that links the two previous tables to ABoVE grid cells using a spatial intersection routine

Please refer to [this Jupyter Notebook (dev/build-cmr-tables.ipynb)](dev/build-cmr-tables.ipynb) if you're interested in learning about the process for building the tables.

## Run to launch the widget:

In [1]:
from ABoVE import *

TESTING


In [ ]:
test = granule_locator_table.loc[granule_locator_table["collection_short_name"]=="ABoVE_Open_Water_Map_1643"]
test

In [ ]:
above

In [ ]:

def update_rendered_granule_table(granule_selection_table):
    """ """

    # make new qgrids
    table_qgrid = get_qgrid(
        granule_selection_table, 
        "granuleid", 
        granule_column_definitions)
    #table_qgrid.on(
    #    'selection_changed', 
    #    handle_dataset_table_select)

    output_containers.selected_index = 1 
    output_granules.clear_output()
    with output_granules:
        display(granules_results_header)
        display(table_qgrid)


def handle_dataset_table_select(event, qgrid_widget):
    """ """

    print(test)
    rownum = qgrid_widget.get_selected_df()
    print(rownum)

    rowdf = qgrid_widget.get_selected_df()
    index = rowdf["title"]
    #granules = granule_locator_table.loc[granule_locator_table["title"]==index]
    output_containers.selected_index = 1 
    with output_granules:
        display(granules_results_header)
        display(index)

    #granules1 = granules[["granuleid","url_datapool"]]

    #update_rendered_granule_table(granules1)


def update_rendered_dataset_table(dataset_selection_table):
    """ """

    # make new qgrids
    table_qgrid = get_qgrid(
        dataset_selection_table, 
        "title", 
        dataset_column_definitions)
    table_qgrid.on(
        'selection_changed', 
        handle_dataset_table_select)

    output_containers.selected_index = 0    
    output_datasets.clear_output()
    with output_datasets:
        display(dataset_results_header)
        display(table_qgrid)



In [ ]:
from ipywidgets import Accordion

In [ ]:
accordion = Accordion(children=[HTML("Datasets"), HTML("Granules")])
accordion.selected_index = None
accordion.set_title(0, 'Slider')
accordion.set_title(1, 'Text')
accordion

In [ ]:
accordion.selected_index = 0

## generate map grid polygon layers

In [ ]:
def get_qgrid(df, index="title"):
    """ """
    
    df.set_index(index, inplace=True)
    tab = qgrid.show_grid(
        df,
        column_definitions={
            "title": {"width": 600},
            "start_time": {"width": 150},
            "end_time": {"width": 150}},
        grid_options={
            'forceFitColumns': False,
            "maxVisibleRows": 8},
        show_toolbar=False)

    return(tab)


# ---------------------------------------------------------------------------

def update_cell_clicked(*args, **kwargs):
    """ """
    draw_control.clear()

    if "properties" in kwargs.keys():
        on = kwargs["properties"]["grid_id"]

        datasets1, granules1, shapelies1 = get_by_tiles([on])
        tab = get_qgrid(datasets1)

        # make layer that represents selected cell and add to selected_layer
        selected_layer.clear_layers()
        x,y = shapelies1[0].exterior.coords.xy
        selected_layer.add_layer(Polygon(locations=list(zip(y,x))))
        centroid = shapelies1[0].centroid
        mapw.center = (centroid.y, centroid.x)

        output.clear_output()
        with output:
            display(dataset_results_header)
            display(tab)


# ---------------------------------------------------------------------------


def update_poly_drawn(*args, **kwargs):
    """ """
  
    draw_control.clear()                       # clear draw, selection layers

    if "geo_json" in kwargs.keys():
       
        drawn_json = kwargs["geo_json"]        # make shapely from geojson 
        shapely_geom = shape(drawn_json["geometry"])
        cells = grid_dict

        # iterate over cells and collect intersecting cells
        on, shapes = [], []
        for id, cell in cells.items():
            if shapely_geom.intersects(cell.shape):
                on.append(id)
                shapes.append(cell.shape)
    
        # get the union of all of the cells that are toggled on
        union = cascaded_union(shapes)
        centroid = union.centroid

        # make layer that represents selected cells and add to selected_layer
        selected_layer.clear_layers()
        x,y = union.exterior.coords.xy
        selected_layer.add_layer(Polygon(locations=list(zip(y,x))))
        mapw.center = (centroid.y, centroid.x)
    
        # get datasets and granules and display table(s)
        datasets1, granules1, shapelies1 = get_by_tiles(on)
        tab = get_qgrid(datasets1)

        output.clear_output()
        with output:
            display(dataset_results_header)
            display(tab)

    else:
        
        print("Nothing happened.")

In [ ]:
# generate map grid polygon layers
grid_layers = LayerGroup()
grid_dict = {}

for feat in above_grid["features"]:
    level = feat["properties"]["grid_level"]
    if level=="B":
        Cell_object = Cell(feat) 
        Cell_object.layer.on_click(update_cell_clicked)
        grid_id = Cell_object.id
        grid_dict[grid_id] = Cell_object
        grid_layers.add_layer(grid_dict[grid_id].layer)

# make an attribute that will hold selected layer
selected_layer = LayerGroup()

mapw = Map(
    layers=(esri, grid_layers, selected_layer, ),
    center=(65, -100), 
    zoom=3, 
    width="auto", 
    height="auto",
    scroll_wheel_zoom=True)

# map draw controls
draw_control = DrawControl()
draw_control.polyline =  {}
draw_control.circle = {}
draw_control.circlemarker = {}
draw_control.remove = False
draw_control.edit = False
draw_control.polygon = {**draw_style}
draw_control.rectangle = {**draw_style}
draw_control.on_draw(update_poly_drawn)
mapw.add_control(draw_control)

# output display
output = Output(layout=Layout(width="auto", height="auto"))

# make the widget layout
ui = VBox([map_header, mapw, output], layout=Layout(width="auto"))

# display ui
display(ui)